<h1>Predicting game prices</h1>
<h5 style="margin-left: 2rem">By: Elad Ben-Haim, Shalev Hadar</h5>
<br/>
<br/>

<h4>נושא המחקר</h4>
<table dir="rtl">
    <tr>
        <th>
            נושא המחקר
        </th>
        <th>
            פירוט הנושא
        </th>
    </tr>
    <tr>
        <td>
        האם ניתן לחזות מחיר של משחק בעוד x זמן?
        </td>
        <td>
        מתי הכי ישתלם לקנות את המשחק בעתיד
        </td>
    </tr>
    <tr>
        <td>
        האם ניתן לחזור מתי למוכר הכי משתלם לעשות מבצע על המשחק?
        </td>
        <td>
        מתי הכי כדאי למוכר לעשות מבצע כדי להביא עוד שחקנים ולהמשיך למכור עם הרווח הגדול ביותר
        </td>
    </tr>
</table>
<br/>

<br/>

<h4>נתוני המחקר ודרכי ניתוח</h4>
<table dir="rtl">
    <tr>
        <th>
            נתוני המחקר & דרכי ניתוח
        </th>
        <th>
             ואיך ננתח אותם
        </th>
    </tr>
    <tr>
        <td>
		פירוט הנתונים: פרטים פיננסיים הוא נמכר בזמן X ואז לחזות בכמה הוא ימכר בזמן Y ובנוסף פרטים על המשחק - כמו שם, ז'אנר פופולריות וכו'.
        </td>
        <td>
            <p style="font-size: 1.1rem">
                נשתמש ב-Crawling על אתר isThereAnyDeal(Fig.3)<br/>
                כדי לא לקבל הודעת שגיאה על שימוש יתר, נשתמש ב-PROXY כדי לא להחסם ע"י isThereAnyDeal<br/>
                ונייבא משם את כל המידע הדרוש כדי לחזות את המחיר של משחק בעוד X זמן מסוים.<br/>
                ראינו שלכל משחק קיים באיזור ה1000+ (Fig.2) רשומות של log (Fig.1) של המחיר שלו ביחס לזמן, ולחנות בה הוא נמכר ואת העלייה\ירידה במחיר של המשחק ביחד ללוג הקודם.<br/>
                בעזרת STEAM API נוציא את ז'אנר המשחק, שנת הייצור ועוד פרטים מורכבים יותר על המשחק עצמו<br/>
                ולבסוף נצרף לכל לוג את פרטי המשחק ונקבל Dataset בגודל n = כמות המשחקים, x = כמות הלוגים, כלומר n*x<br/>
                אנחנו מעוניינים כרגע לקחת את 100 המשחקים הראשונים ולפי מה שראינו כמות הלוגים בדרך כלל היא לפחות 1000 אז נקבל בסביבות ה100,000+ רשומות.<br/>
            </p>
        </td>
    </tr>
    <tr>
        <td>
		דרכי ניתוח: נשתמש בכלים שלמדנו במהלך הקורס לעבד\ללמוד מהנתונים כמה מידע שאנו צריכים למטרה זו
        </td>
        <td>
            <p style="font-size: 1.1rem">
                ננתח את הDataFrame, בעזרת טבלאות יחסי משתנים, סטטיסטיקות, ולבסוף ננסה ללמד מכונה שתחזה את התאריך של המחיר הזול ביותר בשנה מסוימת, ואת המחיר אשר יביא את כמות המכירות הגדול ביותר
            </p>
        </td>
    </tr>
</table>
<br/>
<table style="width:100%;grid-template-rows: 1fr 1fr 1fr;">
<tr>
    <td>
        <figure>
            <img src="images\log_table_for_ds3.png" alt="Is there any deal log table">
            <figcaption>Fig.1 - The main crawled data source</figcaption>
        </figure>
    </td>
    <td>
        <figure>
            <img src="images\number_of_logs_for_ds3.png" alt="Example for number of rows in a typical game">
            <figcaption>Fig.2 - Example for number of rows in a typical game (Dark souls 3) </figcaption>
        </figure>
    </td>
    <td>
        <figure>
            <img src="images\is_there_any_deal_site_example_ds3.png" alt="Is there any deal game page">
            <figcaption>Fig.3 - IsThereAnyDeal game page</figcaption>
        </figure>
    </td>
</tr>

<h4>Importing</h6>

In [16]:
import requests
import bs4
from bs4 import BeautifulSoup
import random
import itertools
import re

<h4>Global functions</h4>

In [3]:
def get_html_response(url: str, proxy: str) -> requests.Response:
    if (proxy is not None):
        return requests.get(url, proxies={"http": proxy, "https":proxy})
    else
        return requests.get(url)

def get_response_as_beautiful_soup(htmlText: requests.Response) -> BeautifulSoup:
    return BeautifulSoup(htmlText.text, 'html.parser')

<br/>

<h4>Defining proxies for scraping</h4>

<h5>Get proxy list response html website</h5>
<h6>get the html as response object instead of getting the html again and again</h6>

In [4]:
# Get the html of the proxy list website
def get_proxy_list_html() -> requests.Response:
    # Website to get free proxies
    return get_html_response('https://free-proxy-list.net/')

In [ ]:
proxies_response = get_proxy_list_html()

<h5>Scrape proxy ip addresses</h5>
<h6>gets the ip addresses as a list, shuffles them and returns an iterator to cycle through when making scrape requests</h6>

In [5]:
def get_proxy_list() -> itertools.cycle:
    soup = get_response_as_beautiful_soup(proxies_response)
    proxy_soup_list = soup.select('#list > div > div.table-responsive > div > table > tbody > tr')
    proxy_list = list(map(lambda i: i.select('td:nth-child(1)')[0].text + ':' + i.select('td:nth-child(2)')[0].text, proxy_soup_list))
    random.shuffle(proxy_list)
    return itertools.cycle(proxy_list)

In [ ]:
proxy_list = get_proxy_list()

In [ ]:
def get_proxied(url: str) -> requests.Response:
    i = 0
    while(i < len(proxy_list) - 1):
        try:
            response = get_html_response(url)
            proxy_list = next(proxy_list)
            return response
        except:
            i += 1

    raise RuntimeError('None of the proxies provided work.')

<br/>

<h4>Scrape isThereAnyDeal website</h4>
<h6>Steps:</h6>
<ol>
<li>Crawl list of top 100 trending games</li>
<li>For each game:</li>
<ul>
    <li>get the game details from steam API using "appId" scraped either from PC Gaming Wiki or Steam Ladder links</li>
    <li>mine price Log history on isThereAnyDeal</li>
    <li>mine Number of sales of the game</li>
<ul>
</ol>

<h5>Get list of 100 top trending games</h5>

In [6]:
def get_is_there_any_deal_games_response() -> requests.Response:
    filteredUrl = 'https://isthereanydeal.com/?by=trending:desc#/filter:&releaseyear/2010/2021,&pl/windows,&drm/steam,steam,-dlc,-type/6,-type/8,-type/7;/options:all'
    return get_proxied(filteredUrl)

In [ ]:
is_there_any_deal_games_response = get_is_there_any_deal_games_response()

In [7]:
def get_game_links():
    soup = get_response_as_beautiful_soup(is_there_any_deal_games_response)
    print(soup.select(""))

In [ ]:
game_links = get_game_links()

In [36]:
def get_game_details_response():
    return get_html_response("https://isthereanydeal.com/game/deadbydaylight/history/#/chart:low")

In [ ]:
game_details_response = get_game_details_response()

In [ ]:
def get_game_details():
    soup = get_response_as_beautiful_soup(game_details_response)
    return soup.select("#pageContainer > script:nth-child(9)")

gamedetails = str(get_game_details()[0])
m = re.findall('JSON\.stringify\((.*?)\)', gamedetails)
# print(m)

<h3>Used Resources</h3>
<dl>
    <dt>Scraping</dt>
    <dd>
        <a href="https://isthereanydeal.com/game/reddeadredemptionii/info/">
            <b>Is-There-Any-Deal website</b> For scraping cost history and more financial details
        </a>
    </dd>
    <dd>
        <a href="https://www.geeksforgeeks.org/web-scraping-without-getting-blocked/">
            <b>Using Proxies to avoiding getting blocked</b>
        </a>
    </dd>
    <dd>
        <a href="https://wiki.teamfortress.com/wiki/User:RJackson/StorefrontAPI#App_info">
            <b>Steam StoreFront API</b> Limited to 100,000 requests per day, and no more than 10 per second
        </a>
    </dd>
    <dd>
        <a href="https://store.steampowered.com/api/appdetails">Steam API for Game Metadata - https://store.steampowered.com/api/appdetails?appids=1091500</a>
    </dd>
</dl>